In [167]:
import pandas as pd
import numpy as np

df = pd.read_csv('pbp_raw/0022300142_pbp.csv',index_col=0)

def str_to_tuple(s):
    return tuple(s.strip('() ').replace(' ', '').split(','))

df['h_lineup'] = df.h_lineup.apply(lambda x: str_to_tuple(x))
df['a_lineup'] = df.a_lineup.apply(lambda x: str_to_tuple(x))

eventdict = {1:'FGM',
             2:'FGA',
             3:'FTA',
             4:'TRB',
             5:'TURNOVER',
             6:'FOUL',
             8:'SUB',
             9:'TIMEOUT',
             10:'JUMP_BALL',
             12:'Q_START',
             13:'Q_END',
             18:'REPLAY'             
            }
cols = ['game_id', 'lineup','player_id','PTS','FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'MIN', 'POSS']


In [168]:
#allll this ugly stuff just to get a column called 'rebound_type'
mask = (df.eventmsgtype == 4) & (~df.homedescription.isnull() | ~df.visitordescription.isnull())
pattern = r'Off:(\d+) Def:(\d+)'

home_extract = df['homedescription'].str.extract(pattern)
df['offensive_rebounds'] = home_extract[0]
df['defensive_rebounds'] = home_extract[1]

visitor_extract = df['visitordescription'].str.extract(pattern)
df['offensive_rebounds'].fillna(visitor_extract[0], inplace=True)
df['defensive_rebounds'].fillna(visitor_extract[1], inplace=True)

df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].fillna(0)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].astype(int)
df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df['rebound_type'] = ''
df.loc[(df.defensive_rebounds > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.drop([ 'offensive_rebounds', 'defensive_rebounds', 'def_reb_previous', 'off_reb_previous'],inplace=True,axis=1)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)

In [169]:
fga_filter = (df.eventmsgtype==1) | (df.eventmsgtype==2)
two_pt_filter = (df.eventmsgtype==1) & ~((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_pt_filter = (df.eventmsgtype==1) & ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_a_filter = ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
ft_filter = (df.eventmsgtype==3) & ((df.homedescription.str.contains('PTS',na=False)) | (df.visitordescription.str.contains('PTS',na=False)))
fta_filter = df.eventmsgtype==3 
assist_filter = (df.eventmsgtype==1) & ~df.player2_name.isnull()

orb_filter = df.rebound_type == 'ORB'
drb_filter = df.rebound_type == 'DRB'
trb_filter = (df.rebound_type == 'DRB') | (df.rebound_type == 'ORB')
stl_filter = df.homedescription.str.contains('STEAL',na=False) | df.visitordescription.str.contains('STEAL',na=False)
blk_filter = df.homedescription.str.contains('BLOCK',na=False) | df.visitordescription.str.contains('BLOCK',na=False)
to_filter = df.eventmsgtype==5
foul_filter = df.eventmsgtype==6
time_filter = (df.play_elapsed_time != 0)

In [170]:
made_last_ft_pattern = r'(1 of 1|2 of 2|3 of 3).*PTS'
visitor_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.homedescription.isna())) | # home team drb means visitor poss ends
((df.eventmsgtype == 4) & (df.eventmsgactiontype==1) & (~df.homedescription.isna())) |
(df.homedescription.str.contains('STEAL',na=False)) | # home team steal means visitor poss ends
((df.eventmsgtype==1) & ~(df.visitordescription.isna())) | # visitor team makes shot means visitor poss ends
((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) | #visitor team MAKES last ft means poss ends
((df.eventmsgtype == 5) & (df.homedescription.isna())) #unforced visitor turnover
) 

home_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.visitordescription.isna())) | # home team drb means visitor poss ends
#((df.eventmsgtype == 4) & (df.eventmsgactiontype==1) & (~df.visitordescription.isna())) |
(df.visitordescription.str.contains('STEAL',na=False)) |
((df.eventmsgtype==1) & ~(df.homedescription.isna())) |
((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |
((df.eventmsgtype == 5) & (df.visitordescription.isna()))
) 


C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_46300\2066321773.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) | #visitor team MAKES last ft means poss ends
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_46300\2066321773.py:16: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |


In [171]:
df['home_poss'],df['away_poss'] = 0,0
df.loc[visitor_poss_filter,'away_poss'] = 1
df.loc[home_poss_filter,'home_poss'] = 1
# Determine the most recent possession
def determine_possession(row):
    if row['home_poss'] == 1:
        return 'home'
    elif row['away_poss'] == 1:
        return 'away'

In [172]:

df['home_poss'],df['away_poss'] = 0,0
df.loc[visitor_poss_filter,'away_poss'] = 1
df.loc[home_poss_filter,'home_poss'] = 1
# Apply the function to each row to create a new 'new_col' column
df['has_ball'] = df.apply(determine_possession, axis=1)

In [173]:

# Fill NaN values in the 'new_col' column based on the previous row
df['has_ball'] = df['has_ball'].fillna(method='bfill')


home_mask = df['has_ball'] == 'home'
home_counter = df.loc[home_mask & (df['has_ball'].shift(-1) == 'away'), 'home_poss'].cumsum()
new_index = range(df.index.min(), df.index.max() + 1)
home_counter = home_counter.reindex(new_index, fill_value=np.NaN).fillna(method='bfill')

# Use the correct DataFrame column to assign values
df.loc[home_mask, 'home_poss'] = home_counter


away_mask = df['has_ball'] == 'away'
away_counter = df.loc[away_mask & (df['has_ball'].shift(-1) == 'home'), 'away_poss'].cumsum()
new_index = range(df.index.min(), df.index.max() + 1)
away_counter = away_counter.reindex(new_index, fill_value=np.NaN).fillna(method='bfill')

# Use the correct DataFrame column to assign values
df.loc[away_mask, 'away_poss'] = away_counter

In [174]:
def get_stat_df(pbp_df,filter,stat_type,stat_value,player_num='1'):
    filtered_df = pbp_df.loc[filter][['game_id','eventnum',f'player{player_num}_id',f'player{player_num}_name','h_lineup','a_lineup','home_poss','away_poss']]
    filtered_df.rename(columns={f'player{player_num}_id': 'player_id',
                        f'player{player_num}_name': 'player_name'},inplace=True )
    filtered_df[stat_type] = stat_value
    return filtered_df

In [175]:
def get_time_credit(df):
    game_id = df.game_id[0]
    box_score = pd.read_csv(f"box_scores/box_00{game_id}.csv", index_col=0)
    box = box_score[["player_id", "player_name"]].values.tolist()
    id_to_name = {id: name for id, name in box}
    play_times = df.groupby(['game_id','home_poss','away_poss','h_lineup','a_lineup'])["play_elapsed_time"].sum().reset_index()
    play_times['play_end_time'] = df.groupby(['game_id','home_poss','away_poss','h_lineup','a_lineup'])["total_elapsed_time"].max().reset_index()['total_elapsed_time']
    h_expl = play_times.explode('h_lineup').reset_index(drop=True)
    h_expl.rename(columns={'h_lineup':'player_id'},inplace=True)

    a_expl = play_times.explode('a_lineup').reset_index(drop=True)
    a_expl.rename(columns={'a_lineup':'player_id'},inplace=True)

    h_expl = h_expl.merge(play_times,on= ['game_id','home_poss','away_poss','a_lineup','play_elapsed_time','play_end_time'])
    a_expl = a_expl.merge(play_times,on= ['game_id','home_poss','away_poss','h_lineup','play_elapsed_time','play_end_time'])
    time_credits = pd.concat((h_expl,a_expl))
    time_credits['player_name'] = time_credits['player_id'].apply(lambda x: id_to_name[int(x)])
    time_credits['player_id'] = time_credits['player_id'].astype(int)
    return time_credits
time_credits = get_time_credit(df)
time_credits.sample(3)

,game_id,home_poss,away_poss,player_id,a_lineup,play_elapsed_time,play_end_time,h_lineup,player_name
1055,22300142,35,0.0,201939,"(201939, 202691, 203110, 203952, 1626172)",22.0,1118.0,"(1627747, 1628378, 1628386, 1629622, 1630596)",Stephen Curry
711,22300142,0,88.0,1630541,"(201939, 202691, 203110, 203952, 1630541)",0.0,2727.0,"(1628378, 1628386, 1629622, 1629636, 1630596)",Moses Moody
1075,22300142,39,0.0,201939,"(201939, 202691, 203110, 203952, 1627780)",0.0,1250.0,"(1627747, 1627777, 1628378, 1629622, 1630596)",Stephen Curry


In [176]:
#results is a list of every stat credit for every play
results = get_stat_df(df,two_pt_filter,'PTS',2)
results =pd.concat((results, get_stat_df(df,three_pt_filter,'PTS',3)))
results =pd.concat((results, get_stat_df(df,ft_filter,'PTS',1)))

results =pd.concat((results, get_stat_df(df,two_pt_filter,'FGM',1)))
results =pd.concat((results, get_stat_df(df,three_pt_filter,'3PM',1)))
results =pd.concat((results, get_stat_df(df,ft_filter,'FTM',1)))

results =pd.concat((results, get_stat_df(df,fga_filter,'FGA',1)))
results =pd.concat((results, get_stat_df(df,fta_filter,'FTA',1)))
results =pd.concat((results, get_stat_df(df,three_a_filter,'3PA',1)))

results =pd.concat((results, get_stat_df(df,trb_filter,'TRB',1)))
results =pd.concat((results, get_stat_df(df,drb_filter,'DRB',1)))
results =pd.concat((results, get_stat_df(df,orb_filter,'ORB',1)))

results =pd.concat((results, get_stat_df(df,stl_filter,'STL',1,player_num=2)))
results =pd.concat((results, get_stat_df(df,blk_filter,'BLK',1,player_num=3)))
results =pd.concat((results, get_stat_df(df,to_filter,'TOV',1,player_num=1)))
results =pd.concat((results, get_stat_df(df,foul_filter,'PF',1)))

results =pd.concat((results, get_stat_df(df,assist_filter,'AST',1,player_num='2')))

#results += apply_stat_to_lineup(df,home_poss_filter,'poss',1,a_or_v='h')#h or a
#results += apply_stat_to_lineup(df,visitor_poss_filter,'poss',1,a_or_v='a')#h or a
#results += get_time_credit_list(df)
time_credits = get_time_credit(df)
results = pd.concat((results,time_credits))
temp = results
results = results.groupby(['game_id','home_poss','away_poss','player_id','player_name','h_lineup','a_lineup']).sum().reset_index()
#results['max_eventnum'] = results.groupby(['game_id','home_poss','away_poss','player_id','player_name','h_lineup','a_lineup'])['eventnum'].max().reset_index(drop=True)
results.loc[(results.player_name=='Stephen Curry')]

,game_id,home_poss,away_poss,player_id,player_name,h_lineup,a_lineup,eventnum,PTS,FGM,...,TRB,DRB,ORB,STL,BLK,TOV,PF,AST,play_elapsed_time,play_end_time
10,22300142,0,1.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0
20,22300142,0,2.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,62.0
30,22300142,0,3.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,81.0
40,22300142,0,4.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,118.0
50,22300142,0,5.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",29.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9.0,139.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2383,22300142,84,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.0,2649.0
2393,22300142,85,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2681.0
2404,22300142,86,0.0,201939,Stephen Curry,"(1627747, 1628378, 1628386, 1629622, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,2711.0
2405,22300142,86,0.0,201939,Stephen Curry,"(1628378, 1628386, 1629622, 1629636, 1630596)","(101108, 201939, 202691, 203110, 203952)",0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2711.0


In [177]:
df.columns

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'pctimestring', 'homedescription', 'neutraldescription',
       'visitordescription', 'score', 'scoremargin', 'player1_id',
       'player1_name', 'player1_team_abbreviation', 'player2_id',
       'player2_name', 'player2_team_abbreviation', 'player3_id',
       'player3_name', 'player3_team_abbreviation', 'timeinseconds',
       'play_elapsed_time', 'total_elapsed_time', 'max_time', 'time_remaining',
       'h_lineup', 'a_lineup', 'rebound_type', 'home_poss', 'away_poss',
       'has_ball'],
      dtype='object')

In [178]:
game_id = df.game_id[0]
box_score = pd.read_csv(f"box_scores/box_00{game_id}.csv", index_col=0)
box = box_score[["player_id", "player_name"]].values.tolist()
id_to_name = {id: name for id, name in box}
play_times = df.groupby(['game_id','home_poss','away_poss','h_lineup','a_lineup'])["play_elapsed_time"].sum().reset_index()

In [179]:
play_times.loc[play_times.home_poss==1]

,game_id,home_poss,away_poss,h_lineup,a_lineup,play_elapsed_time
123,22300142,1,0.0,"(1628378, 1628386, 1629622, 1629636, 1630596)","(201939, 202691, 203110, 203952, 1626172)",27.0


In [180]:
len(results.loc[results.player_name=='Stephen Curry'])

163

In [181]:
box = pd.read_csv(f"box_scores/box_0022300142.csv", index_col=0)
away_team_name = box['team_abbreviation'].iloc[0]
home_team_name = box['team_abbreviation'].iloc[-1]
opp_team_dict = {away_team_name:home_team_name,
                 home_team_name:away_team_name}
team_to_id_list = box[['team_abbreviation','player_id']].values
player_team_dict = {player_id:team for (team,player_id) in team_to_id_list}
results['team'] = results['player_id'].apply(lambda x: player_team_dict[x])
results['opp'] = results['team'].apply(lambda x: opp_team_dict[x])


In [182]:
results['team'] = results['player_id'].apply(lambda x: player_team_dict[x])
results['opp'] = results['team'].apply(lambda x: opp_team_dict[x])
results['H/A'] = results['team'].apply(lambda x: 1 if x == home_team_name else 0)

In [185]:
results['off_poss'] = np.NaN
results['def_poss'] = np.NaN
results.loc[((results['H/A']==1) & (results.home_poss != 0)), 'off_poss'] = results['home_poss']  # player is on home team, home team has ball, therefore offensive
results.loc[((results['H/A']==0) & (results.away_poss != 0)), 'off_poss'] = results['away_poss']  # player is on visitor team, visitor team has ball, therefore offensive

results.loc[((results['H/A']==1) & (results.away_poss != 0)), 'def_poss'] = results['away_poss']  #player is on home team, away team has ball, therefore defensive
results.loc[((results['H/A']==0) & (results.home_poss != 0)), 'def_poss'] = results['home_poss']  #player is on away team, home team has ball, therefore defensive
results['off_poss'].fillna(0,inplace=True)
results['def_poss'].fillna(0,inplace=True)
results = results[['game_id','player_id', 'player_name','play_elapsed_time','play_end_time', 'team', 'opp', 'H/A', 'off_poss', 'def_poss',
       'h_lineup', 'a_lineup', 'FGM', '3PM', 'FTM', 'FGA',
       'FTA', '3PA', 'TRB', 'DRB', 'ORB', 'STL', 'BLK', 'TOV', 'PF', 'AST','PTS'
       ]]#dropping eventnum, home_poss,away_poss as well as reordering

In [187]:
results.loc[(results.player_name=='Stephen Curry') & (results.off_poss != 0)].sort_values(by='play_end_time')

,game_id,player_id,player_name,play_elapsed_time,play_end_time,team,opp,H/A,off_poss,def_poss,...,3PA,TRB,DRB,ORB,STL,BLK,TOV,PF,AST,PTS
10,22300142,201939,Stephen Curry,20.0,20.0,GSW,CLE,0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,22300142,201939,Stephen Curry,15.0,62.0,GSW,CLE,0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,22300142,201939,Stephen Curry,13.0,81.0,GSW,CLE,0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,22300142,201939,Stephen Curry,22.0,118.0,GSW,CLE,0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,22300142,201939,Stephen Curry,9.0,139.0,GSW,CLE,0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,22300142,201939,Stephen Curry,0.0,13635.0,GSW,CLE,0,88.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1164,22300142,201939,Stephen Curry,0.0,13635.0,GSW,CLE,0,88.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1165,22300142,201939,Stephen Curry,0.0,13635.0,GSW,CLE,0,88.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1166,22300142,201939,Stephen Curry,0.0,13635.0,GSW,CLE,0,88.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
